# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on Structured Streaming (QueryListener & Output sinks)** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Spark Session creation

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Create FileStream

In [ ]:
traffic_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/traffic_data/input/")

### Extract traffic info

In [ ]:
from pyspark.sql.functions import split

traffic_df = traffic_lines.select(split(traffic_lines.value, " ").alias("traffic_array"))
traffic_df = traffic_df.withColumn("date", traffic_df["traffic_array"].getItem(0).cast("date"))
traffic_df = traffic_df.withColumn("time", traffic_df["traffic_array"].getItem(1).cast("timestamp"))
traffic_df = traffic_df.withColumn("vehicle_id", traffic_df["traffic_array"].getItem(2).cast("string"))
traffic_df = traffic_df.withColumn("speed", traffic_df["traffic_array"].getItem(3).cast("integer"))
traffic_df = traffic_df.withColumn("position", traffic_df["traffic_array"].getItem(4).cast("string"))

traffic_df.printSchema()

### Extract vehicles that exceed 80 KM/H

In [ ]:
traffic_df = traffic_df.filter(traffic_df.speed > 80)

### Set Stream's sink

In [ ]:

query = traffic_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='2 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)

In [ ]:
query_files = traffic_df \
                .writeStream \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/traffic_data/output/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(10)

### Verify data

In [ ]:
speed_df = spark \
            .read \
            .parquet("/home/jovyan/notebooks/data/traffic_data/output/")

speed_df.show(4, False)

In [79]:
sc.stop()